# Prediction of daily monsoon rainfall

Two files are attached which contain daily rainfall data over India for 2010 and 2011. Both of them contain a 357x122 matrix (XR1 and XR) an a binary vector (ZR1 and ZR). The matrices contain rainfall amounts at 357 locations over India, on each day during the monsoon seasons of 2010 and 2011 (122 days from 1 June to 30 September). ZR1 and ZR are binary vectors which classify every day as 'rainy" (1) or non-rainy (0) based on the rainfall across the landmass.

1) Read the .mat files in Python and access the variables

2) Use a linear regression model to predict the rainfall XR(s,t) at any location 's' on day 't', using as predictor the rainfall at all other locations on the same day, and also rainfall at the same location on the previous 2 days [XR(1,t)....XR(s-1,t), XR(s+1,t),....XR(357,t), XR(s,t-1), XR(s,t-2)]. Use 2010 data for training.

Build such a model for s=42 (Mumbai), s=158 (Delhi), s= 299 (Kharagpur)

3) Use the same model to predict the rainfall at these 3 locations on each day of 2011.  Use values in XR as predictors. Compare the results with the true values and compute error for 3 locations separately.

4) Repeat the same process using LASSO linear regression. Using the coefficients, identify the top 5 predictors for each of the 3 locations.

5) Use Decision Tree on 2010 data to classify each day as 1 or 0. For each day, use the 357-dimensional rainfall vector as feature vector. Report the 10 most discriminative features (i.e. locations)

6) Use this Decision Tree to classify each day of 2011 as 1 or 0. Report accuracy.

In [59]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, accuracy_score
from math import sqrt
from sklearn.tree import DecisionTreeClassifier

In [60]:
# Loading data
train = scipy.io.loadmat("2010rainfall.mat")
test = scipy.io.loadmat("2011rainfall.mat")
# row 158 is Delhi
# Kgp 299
# row 122 mumbai
# 122 days monsoon

In [61]:
# Extracting required data from matlab variable
train_data = train["XR1"]
test_data = test["XR"]

In [62]:
test_data.shape

(357, 122)

In [63]:
train

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Aug 23 12:22:43 2019',
 '__version__': '1.0',
 '__globals__': [],
 'XR1': array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.46729854, 10.92762184, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 'ZR1': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0

In [64]:
def get_data(x, column, row):
    '''
    Given x (all train data) and specific column and row, it makes a feature vector to be used 
    for that column and row by he rainfall at all other locations on the same day, and also 
    rainfall at the same location on the previous 2 days
    '''
    rows_include = list(range(0, row)) + list(range(row+1, x.shape[0]))
    data = x[rows_include, column]
    data = np.append(data, [x[row-2][column], x[row-1][column]])
    return data

In [65]:
def feature_vector(x, row):
    '''
    Given x (all train data), with the help of get_data function (defined in cell above) it makes a feature vector
    by iterating on all the columns for a particular row (A particular row here represents data collected from a city)
    '''
    x_train = []
    y_train = []
    for i in range(2, x.shape[1]):
        x_train.append(get_data(x, i, row))
        y_train.append(x[row][i])
    return np.array(x_train), np.array(y_train)

In [66]:
x_train, y_train = feature_vector(train_data, 42)

In [67]:
# checking feature_vector function by printing shape of feature vector returned
print(f"x_train shape is {x_train.shape}")
print(f"y_train shape is {y_train.shape}")

x_train shape is (120, 358)
y_train shape is (120,)


## Linear Regression

### Mumbai

In [68]:
# Getting feature vector for Mumbai
row = 42
x_train, y_train = feature_vector(train_data, row)
x_test, y_test = feature_vector(test_data, row)

In [15]:
# Training model
m = LinearRegression()
m.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
# Calcularing Error (RMSE in this case)
error = sqrt(mean_squared_error(m.predict(x_test), y_test))
print(f"Error is {error}")

Error is 25.779489035763323


### Delhi

In [17]:
# Getting feature vector for Delhi
row = 158
x_train, y_train = feature_vector(train_data, row)
x_test, y_test = feature_vector(test_data, row)

In [18]:
# Training model
m = LinearRegression()
m.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
# Calcularing Error (RMSE in this case)
error = sqrt(mean_squared_error(m.predict(x_test), y_test))
print(f"Error is {error}")

Error is 13.399315105760785


### Kharagpur

In [20]:
# Getting feature vector for Kharagpur
row = 299
x_train, y_train = feature_vector(train_data, row)
x_test, y_test = feature_vector(test_data, row)

In [21]:
# Training model
m = LinearRegression()
m.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [69]:
# Calcularing Error (RMSE in this case)
error = sqrt(mean_squared_error(m.predict(x_test), y_test))
print(f"Error is {error}")

Error is 25.3835019883162


## Lasso

### Mumbai

In [23]:
# Getting feature vector for Mumbai
row = 42
x_train, y_train = feature_vector(train_data, row)
x_test, y_test = feature_vector(test_data, row)

In [24]:
# Training model
m = Lasso(max_iter=10000)
m.fit(x_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [70]:
# Calcularing Error (RMSE in this case)
error = sqrt(mean_squared_error(m.predict(x_test), y_test))
print(f"Error is {error}")

Error is 25.3835019883162


In [26]:
coefficients = m.coef_

In [27]:
coefficients.shape

(358,)

In [28]:
# getting top 5 elments from coefficeints
index = sorted(range(len(coefficients)), key=lambda i: coefficients[i], reverse=True)[:5]
print(f"Most important 5 indices are {index}")

Most important 5 indices are [26, 25, 47, 90, 92]


In [29]:
coefficients[26]

0.5790623596033783

In [30]:
coefficients[25]

0.3533835636785544

### Delhi

In [31]:
# Getting feature vector for Delhi
row = 158
x_train, y_train = feature_vector(train_data, row)
x_test, y_test = feature_vector(test_data, row)

In [32]:
# Training model
m = Lasso(max_iter=10000)
m.fit(x_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [33]:
# Calcularing Error (RMSE in this case)
error = sqrt(mean_squared_error(m.predict(x_test), y_test))
print(f"Error is {error}")

Error is 12.208246968097699


In [34]:
coefficients = m.coef_

In [35]:
coefficients.shape

(358,)

In [36]:
# getting top 5 elments from coefficeints
index = sorted(range(len(coefficients)), key=lambda i: coefficients[i], reverse=True)[:5]
print(f"Most important 5 indices are {index}")

Most important 5 indices are [131, 202, 153, 74, 73]


### Kharagpur

In [37]:
# Getting feature vector for Kharagpur
row = 42
x_train, y_train = feature_vector(train_data, row)
x_test, y_test = feature_vector(test_data, row)

In [38]:
# Training model
m = Lasso(max_iter=10000)
m.fit(x_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [39]:
# Calcularing Error (RMSE in this case)
error = sqrt(mean_squared_error(m.predict(x_test), y_test))
print(f"Error is {error}")

Error is 25.3835019883162


In [73]:
# Getting feature importance vcoefficients
coefficients = m.coef_

In [41]:
coefficients.shape

(358,)

In [42]:
# getting top 5 elments from coefficeints
index = sorted(range(len(coefficients)), key=lambda i: coefficients[i], reverse=True)[:5]
print(f"Most important 5 indices are {index}")

Most important 5 indices are [26, 25, 47, 90, 92]


Note that important features are different for different cities as expected

## Decision Tree Classifier

In [72]:
# Getting training and testing data
x_train = train_data.T
y_train = train["ZR1"].T
x_test = test_data.T
y_test = test["ZR"].T

In [44]:
print(f"x_train shape is {x_train.shape}")
print(f"y_train shape is {y_train.shape}")
print(f"x_test shape is {x_test.shape}")
print(f"y_test shape is {y_test.shape}")

x_train shape is (122, 357)
y_train shape is (122, 1)
x_test shape is (122, 357)
y_test shape is (122, 1)


In [45]:
# Training model
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [46]:
# Calculating Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy is {acc}")

Accuracy is 0.7049180327868853


In [71]:
# Getting feature importances
feature_importance = clf.feature_importances_

In [49]:
# Picking the values for feature importance
index = sorted(range(len(feature_importance)), key=lambda i: feature_importance[i], reverse=True)[:10]
print(f"Most important 10 indices are {index}")

Most important 10 indices are [184, 252, 162, 26, 204, 159, 346, 208, 115, 0]
